I used LightGBM as a first attempt to submit.
Not much EDA or Feature engineering, I only did encode the categoricals.
I think there are so many things for improvment, as I am just new to Kaggle.
Hope you guys like it, and please upvote it!

In version 1, I simply copy the way I did in very simple LGBM but with 10 folds CV.

In version 2, I added categorical feature, 9t1 and 8t1 and continuous feature cont95 and cont80

In version 3, I tuned parameters using Optuna for the new features.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import chi2
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import mutual_info_classif
import seaborn as sns
import matplotlib.pyplot as plt
import lightgbm as lgb

In [ ]:
train = pd.read_csv('../input/tabular-playground-series-feb-2021/train.csv')
test = pd.read_csv('../input/tabular-playground-series-feb-2021/test.csv')
sample_sub = pd.read_csv('../input/tabular-playground-series-feb-2021/sample_submission.csv')

In [ ]:
train.head()

In [ ]:
delete_columns = ['id']
train.drop(delete_columns, axis=1, inplace=True)
test.drop(delete_columns, axis=1, inplace=True)

In [ ]:
continuous_features = [i for i in train.columns if 'cont' in i]
continuous = train.filter(items=continuous_features)
## a lot of useful info are simple functions in pandas, like corr
corr_mat= continuous.corr();

mask = np.triu(np.ones_like(corr_mat, dtype=bool))

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(13,10));

# Generate a custom diverging colormap
cmap = sns.diverging_palette(230, 20, as_cmap=True);

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr_mat, mask=mask, cmap=cmap, vmax=.7, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5}, annot=True);

In [ ]:

for c in train.columns:
    if train[c].dtype == 'object':
        lbl = LabelEncoder()
        lbl.fit(list(train[c].values)+list(test[c].values))
        train[c] = lbl.transform(train[c].values)
        test[c] = lbl.transform(test[c].values)
        
display(train.head())


In [ ]:
train['my9t1'] = train['cat9']*train['cat1']
train['my8t1'] = train['cat8']*train['cat1']
train['mycont95'] = train['cont9']*train['cont5']
train['mycont80'] = train['cont8']*train['cont0']

test['my9t1'] = test['cat9']*test['cat1']
test['my8t1'] = test['cat8']*test['cat1']
test['mycont95'] = test['cont9']*test['cont5']
test['mycont80'] = test['cont8']*test['cont0']

In [ ]:
categorical_features = ['cat0', 'cat1', 'cat2', 'cat3', 'cat4', 'cat5', 'cat6','cat7', 'cat8', 'cat9','my9t1', 'my8t1']

In [ ]:
train.head()

In [ ]:
#delete_columns = ['cat0', 'cat1', 'cat2', 'cat3', 'cat4', 'cat5', 'cat6', 'cat7', 'cat8', 'cat9']
y_train = train['target']
X_train = train.drop('target', axis = 1)
X_test = test

In [ ]:
X_train.columns.values

In [ ]:
y_train.shape

In [ ]:
y_preds = []
models = []
oof_train = np.zeros(len(X_train))
cv = KFold(n_splits=10, shuffle=True, random_state=0)

params = {
    'random_state':42,
    'metric': 'rmse',
    'n_jobs': -1,
    #'cat_feature': [x for x in range(len(categorical_features))],
    'bagging_seed':42,
    'feature_fraction_seed':42,
        
    'learning_rate': 0.003595159587476034,
    'max_depth': 33,
    'num_leaves': 90,
    'reg_alpha': 4.859124490669792,
    'reg_lambda': 0.4912355149679248,
    'colsample_bytree': 0.2423175654211017,
    'min_child_samples': 166,
    'subsample_freq': 8,
    'subsample': 0.6257984266472807,
    'max_bin': 800,
    'min_data_per_group': 83,
    'cat_smooth': 24,
    'cat_l2': 13,
    
    'verbose':-1,
    'force_row_wise':'true',
}

y_preds = 0
mses = 0
X_train_columns = X_train.columns.values
feature_importance_df = pd.DataFrame()


for fold_id, (train_index, valid_index) in enumerate(cv.split(X_train, y_train)):
    X_tr = X_train.loc[train_index, :]
    X_val = X_train.loc[valid_index, :]
    y_tr = y_train[train_index]
    y_val = y_train[valid_index]
    
    lgb_train = lgb.Dataset(X_tr, y_tr, categorical_feature=categorical_features)
    lgb_eval = lgb.Dataset(X_val, y_val, reference=lgb_train, categorical_feature=categorical_features)
    
    
    model = lgb.train(params, lgb_train,
                      valid_sets=[lgb_train, lgb_eval],
                      categorical_feature = categorical_features,
                      verbose_eval = 1000,
                      num_boost_round = 30000,
                      early_stopping_rounds=1000,)
    
    fold_importance_df = pd.DataFrame()
    fold_importance_df["Feature"] = X_train_columns
    fold_importance_df["importance"] = model.feature_importance()
    fold_importance_df["fold"] = fold_id + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    
    y_pred = model.predict(X_test, num_iteration=model.best_iteration)
    y_preds += y_pred/10
    models.append(model)
    mse = mean_squared_error(y_val, model.predict(X_val,num_iteration=model.best_iteration))
    mses += mse/10
    print(mse)
    
print(mses)

In [ ]:
#pd.DataFrame(oof_train).to_csv('oof_train_kfold.csv', index=False)

In [ ]:
cols = (feature_importance_df[["Feature", "importance"]]
        .groupby("Feature")
        .mean()
        .sort_values(by="importance", ascending=False)[:3014].index)
best_features = feature_importance_df.loc[feature_importance_df.Feature.isin(cols)]

plt.figure()
sns.barplot(x="importance", y="Feature", data=best_features.sort_values(by="importance",ascending=False))
plt.title('LightGBM Features (averaged over folds)')
plt.tight_layout()

In [ ]:
y_preds = pd.DataFrame(y_preds)

In [ ]:
y_subs = y_preds

In [ ]:
sample_sub['target'] = y_subs
sample_sub.to_csv('submission_CV.csv', index=False)